In [124]:
# Libraries

import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.display import display
from io import StringIO
import csv
import gzip
from unidecode import unidecode
import html5lib
import itertools
from dotenv import load_dotenv
import os
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas


In [126]:
# Load environment variables from .env file
env_path = os.path.abspath("credentials.env")

# Load explicitly
load_dotenv(dotenv_path=env_path, override=True)

print("User:", os.getenv("SNOWFLAKE_USER"))
print("Password:", os.getenv("SNOWFLAKE_PASSWORD"))
print("Account:", os.getenv("SNOWFLAKE_ACCOUNT"))
print("Warehouse:", os.getenv("SNOWFLAKE_WAREHOUSE"))
print("Database:", os.getenv("SNOWFLAKE_DATABASE"))
print("Schema:", os.getenv("SNOWFLAKE_SCHEMA"))
print("Role:", os.getenv("SNOWFLAKE_ROLE"))

# Access Snowflake credentials from environment variables
conn_params = {
    'user': os.getenv('SNOWFLAKE_USER'),
    'password': os.getenv('SNOWFLAKE_PASSWORD'),
    'account': os.getenv('SNOWFLAKE_ACCOUNT'),
    'warehouse': os.getenv('SNOWFLAKE_WAREHOUSE'),
    'database': os.getenv('SNOWFLAKE_DATABASE'),
    'schema': os.getenv('SNOWFLAKE_SCHEMA'),
    'role': os.getenv('SNOWFLAKE_ROLE'),
}  

conn = snowflake.connector.connect(**conn_params)
print(type(conn)) 

User: ANDREALEONEL
Password: Buy100toasters
Account: xf30579.eu-west-2.aws
Warehouse: COMPUTE_WH
Database: BIGBROTHERBRASIL
Schema: WIKIPEDIA
Role: ACCOUNTADMIN
<class 'snowflake.connector.connection.SnowflakeConnection'>


In [127]:
def nominations_scrape(url):

    # Fetch the url content
    response = requests.get(url)

    # Parse the url content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html5lib')
    def remove_accents(text):
        return unidecode(text) if isinstance(text, str) else text

    # Iterate through all text elements in the HTML and replace accents
    for element in soup.find_all(string=True):
        element.replace_with(remove_accents(element))  # Replace text with de

    # Find the h2 with id="Histórico"
    h2_header = soup.find('h2', {'id': 'Histórico'})
    desired_table = None

    if h2_header:
        parent_div = h2_header.find_parent('div')
        next_div = parent_div.find_next_sibling()

    if next_div:
        if next_div.name == "table":
            desired_table = next_div
        else:
            desired_table = next_div.find("table", recursive=False)
    
    # Normalising headers
    if desired_table:

        #Parsing html table to DataFrame
        html_to_table = pd.read_html(StringIO(str(desired_table)))
        Nominations_raw = html_to_table[0]

        # Dynamically extract all column header levels into separate lists
        all_levels = [Nominations_raw.columns.get_level_values(level).tolist() for level in range(Nominations_raw.columns.nlevels)]

        # Merging these lists into one, creating column names that contain the information from the following levels, if any
        headers = []

        for items in zip(*all_levels): # Creates tuples for each column with the different levels.
            merged_items = []
            for i in range(len(items)): # For each item in each tuple
                if i == 0 or items[i] != items[i - 1]: # If it's the first item or if the item is different from the previous item.
                    merged_items.append(items[i]) # Include the item
                else:
                    merged_items.append("")  # Otherwise, add an empty string for duplicates
            headers.append(" - ".join(filter(None, merged_items)))  # Combine non-empty values

        #Set new headers
        headers = ["" if "Unnamed" in item else item for item in headers] # Removing the "Unnamed" levels in the header (column 0)
        Nominations_raw.columns = headers

    # Separate the 3 tables.

    # Replace empty cells with nulls
    Nominations_raw.replace(r'^\s*$', None, regex=True, inplace=True)
    Nominations_raw.replace(r'(nenhum)', None, regex=True, inplace=True)

    # Identify rows where all columns are null - the dividers
    divider_index = Nominations_raw[Nominations_raw.isnull().all(axis=1)].index

    # Split the DataFrame into three parts based on the dividing row indices and add the headers created above
    df_part1 = Nominations_raw.iloc[:divider_index[0]]  # From start to first blank row
    df_part1.columns = headers
    df_part2 = Nominations_raw.iloc[divider_index[0]+1:divider_index[1]]  # Between the blank rows
    df_part2.columns = headers
    df_part3 = Nominations_raw.iloc[divider_index[1]+1:]
    df_part3.columns = headers

    # Storing the individual tables into dataframes
    Nominations = pd.DataFrame(df_part1)
    Individual_nominations = pd.DataFrame(df_part2)
    Eviction_results = pd.DataFrame(df_part3)

    # Manipulating the Nominations table

    # Remove Na columns
    Nominations = Nominations.dropna(axis=1, how='all')

    # Drop any duplicate columns to the index
    Nominations = Nominations.loc[:, ~Nominations.T.duplicated()]

    #Transpose the table
    Nominations = Nominations.T
    Nominations.columns = Nominations.iloc[0]
    Nominations = Nominations[1:]

    # Adding the year of the current file
    Nominations['Edicao'] = url.rsplit('_', 1)[-1]

    # Reset the index to make 'Semana' a column
    Nominations = Nominations.reset_index()

    # Create a new column 'Dinamica' based on what's after the delimiter
    Nominations['Dinamica'] = Nominations['index'].apply(lambda x: x.split(' - ')[1] if ' - ' in x else None)

    # Update 'Semana' to keep only the part before the delimiter
    Nominations['index'] = Nominations['index'].apply(lambda x: x.split(' - ')[0] if ' - ' in x else x)
    Nominations.rename(columns={'index': 'Semana'}, inplace=True)

    # Removing spaces from column names
    Nominations.columns = [col.replace(' ', '_') for col in Nominations.columns]
    Nominations.columns = [col.replace('(', '') for col in Nominations.columns]
    Nominations.columns = [col.replace(')', '') for col in Nominations.columns]


    # Manipulating the Individual_nominations table

    # Remove Na columns
    Individual_nominations = Individual_nominations.dropna(axis=1, how='all')

    #Transpose the table
    Individual_nominations = Individual_nominations.T
    Individual_nominations.columns = Individual_nominations.iloc[0]
    Individual_nominations = Individual_nominations[1:]

    # Drop any duplicate columns
    Individual_nominations = Individual_nominations.loc[:, ~Individual_nominations.columns.duplicated()]

    # Adding the year of the current file
    Individual_nominations['Edicao'] = url.rsplit('_', 1)[-1]

    # Reset the index to make 'Semana' a column
    Individual_nominations = Individual_nominations.reset_index()

    # Create a new column 'Dinamica' based on what's after the delimiter
    Individual_nominations['Dinamica'] = Individual_nominations['index'].apply(lambda x: x.split(' - ')[1] if ' - ' in x else None)

    # Update 'Semana' to keep only the part before the delimiter
    Individual_nominations['index'] = Individual_nominations['index'].apply(lambda x: x.split(' - ')[0] if ' - ' in x else x)
    Individual_nominations.rename(columns={'index': 'Semana'}, inplace=True)

    # Removing spaces from column names
    Individual_nominations.columns = [col.replace(' & ', '_') for col in Individual_nominations.columns]
    Individual_nominations.columns = [col.replace(' ', '_') for col in Individual_nominations.columns]
    Individual_nominations.columns = [col.replace('(', '') for col in Individual_nominations.columns]
    Individual_nominations.columns = [col.replace(')', '') for col in Individual_nominations.columns]
    Individual_nominations.columns = [col.replace('/', '_') for col in Individual_nominations.columns]
    Individual_nominations.columns = [col.replace('[b]', '') for col in Individual_nominations.columns]


    # Manipulating the Eviction_results table

    # Remove Na columns
    Eviction_results = Eviction_results.dropna(axis=1, how='all')
    
    # Drop any duplicate columns to the index
    Eviction_results = Eviction_results.loc[:, ~Eviction_results.T.duplicated()]

    #Transpose the table
    Eviction_results_t = Eviction_results.T
    Eviction_results_t.columns = Eviction_results_t.iloc[0]
    Eviction_results = Eviction_results_t[1:]

    # Drop any duplicate columns to the index
    Eviction_results = Eviction_results.loc[:, ~Eviction_results.T.duplicated()]

    # Adding the year of the current file
    Eviction_results['Edicao'] = url.rsplit('_', 1)[-1]

    # Reset the index to make 'Semana' a column
    Eviction_results = Eviction_results.reset_index()

    # Create a new column 'Dinamica' based on what's after the delimiter
    Eviction_results['Dinamica'] = Eviction_results['index'].apply(lambda x: x.split(' - ')[1] if ' - ' in x else None)

    # Update 'Semana' to keep only the part before the delimiter
    Eviction_results['index'] = Eviction_results['index'].apply(lambda x: x.split(' - ')[0] if ' - ' in x else x)
    Eviction_results.rename(columns={'index': 'Semana'}, inplace=True)

    # Removing spaces from column names
    Eviction_results.columns = [col.replace(' & ', '_') for col in Eviction_results.columns]
    Eviction_results.columns = [col.replace(' ', '_') for col in Eviction_results.columns]
    Eviction_results.columns = [col.replace('(', '') for col in Eviction_results.columns]
    Eviction_results.columns = [col.replace(')', '') for col in Eviction_results.columns]
    Eviction_results.columns = [col.replace('%', 'Porcent') for col in Eviction_results.columns]
    Eviction_results.columns = [col.replace('/', '_') for col in Eviction_results.columns]
    Eviction_results.columns = [col.replace('[b]', '') for col in Eviction_results.columns]  

    # Normalising column names
    for col in Eviction_results.columns:
        if col.startswith(('Outras_Porcent', 'Outras_Porcent_ou_Pontos', 'Outras_Porcent_ou_Votos','Outras_Porcent_ou_Votos')):
            Eviction_results.rename(columns={col: 'porcent_outros_'}, inplace=True)

    # Make duplicate column names unique
    excluded = Eviction_results.columns[~Eviction_results.columns.duplicated(keep=False)]
    counters = {}

    def ren(name):
        if name in excluded:
            return name
        if name not in counters:
            counters[name] = itertools.count()  # Create a new counter
        return f"{name}{next(counters[name])}"

    Eviction_results.columns = [ren(name) for name in Eviction_results.columns]

    # Handling duplicated values in Eliminado
    prefixes = ('Eliminado','porcent_outros_','Expulso','Paredao')

    for prefix in prefixes:
        columns_to_check = [col for col in Eviction_results.columns if col.startswith(prefix)]
        if columns_to_check:
            for index, row in Eviction_results.iterrows():
                unique_values = list(dict.fromkeys(row[columns_to_check]))
                for i, col in enumerate(columns_to_check):
                    Eviction_results.loc[index, col] = unique_values[i] if i < len(unique_values) else None

    Eviction_results = Eviction_results.dropna(axis=1, how='all')
    year = url.rsplit('_', 1)[-1]

    #Nominations.to_csv(f'nominations{year}')
    #Individual_nominations.to_csv(f'individualnominations{year}')
    #Eviction_results.to_csv(f'evictionresults{year}')

    return Nominations, Individual_nominations, Eviction_results
    

In [130]:
# List of URLs to process
base_url = "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_"
number_of_shows = 25

urls = [f"{base_url}{i}" for i in range(1, number_of_shows + 1)]


# Define function to create tables dynamically
def create_table_if_not_exists(conn, table_name, df):
    type_mapping = {
        "int64": "INTEGER",
        "float64": "FLOAT",
        "bool": "BOOLEAN",
        "datetime64[ns]": "TIMESTAMP",
        "object": "VARCHAR"
    }

    # Force mixed-type columns to be string (VARCHAR) in SQL
    for col in df.columns:
        if str(df.dtypes[col]) == "object":
            df[col] = df[col].astype(str)
    
    columns = ", ".join([f"{col} {type_mapping.get(str(dtype), 'VARCHAR')}" for col, dtype in df.dtypes.items()])
    create_table_sql = f"CREATE TABLE IF NOT EXISTS BIGBROTHERBRASIL.WIKIPEDIA.{table_name} ({columns});"
    print(create_table_sql)

    with conn.cursor() as cursor:
        cursor.execute(create_table_sql)
    
    print(f"Table {table_name} created (or already exists).")

# Define function to insert data into Snowflake
def upload_to_snowflake(conn, df, table_name):
    success, nchunks, nrows, _ = write_pandas(
        conn, df,
        table_name=table_name,
        database="BIGBROTHERBRASIL",
        schema="WIKIPEDIA",
        quote_identifiers=False
    )

    if success:
        print(f"Successfully uploaded {nrows} rows to {table_name}")
    else:
        print(f"Upload failed for {table_name}")

# Iterate through URLs and process each one
for i, url in enumerate(urls, start=1):
    
    # Scrape the data
    nominations_df, individual_nominations_df, eviction_results_df = nominations_scrape(url)

    # Dynamically generate table names
    tables = {
        f"NOMINATIONS_{i}": nominations_df,
        f"INDIVIDUAL_NOMINATIONS_{i}": individual_nominations_df,
        f"EVICTIONS_{i}": eviction_results_df,
    }

    # Create tables and insert data
    for table_name, df in tables.items():
        create_table_if_not_exists(conn, table_name, df)
        upload_to_snowflake(conn, df, table_name)  

CREATE TABLE IF NOT EXISTS BIGBROTHERBRASIL.WIKIPEDIA.NOMINATIONS_1 (Semana VARCHAR, Lider VARCHAR, Indicado_Lider VARCHAR, Indicado_Casa VARCHAR, Edicao VARCHAR, Dinamica VARCHAR);
Table NOMINATIONS_1 created (or already exists).
Successfully uploaded 9 rows to NOMINATIONS_1
CREATE TABLE IF NOT EXISTS BIGBROTHERBRASIL.WIKIPEDIA.INDIVIDUAL_NOMINATIONS_1 (Semana VARCHAR, Kleber VARCHAR, Vanessa VARCHAR, Andre VARCHAR, Sergio VARCHAR, Alessandra VARCHAR, Estela VARCHAR, Adriano VARCHAR, Helena VARCHAR, Cristiana VARCHAR, Bruno VARCHAR, Xaiane VARCHAR, Caetano VARCHAR, Edicao VARCHAR, Dinamica VARCHAR);
Table INDIVIDUAL_NOMINATIONS_1 created (or already exists).
Successfully uploaded 11 rows to INDIVIDUAL_NOMINATIONS_1
CREATE TABLE IF NOT EXISTS BIGBROTHERBRASIL.WIKIPEDIA.EVICTIONS_1 (Semana VARCHAR, Notas VARCHAR, Berlinda__Paredao VARCHAR, Eliminado0 VARCHAR, Eliminado1 VARCHAR, Eliminado2 VARCHAR, porcent_outros_ VARCHAR, Edicao VARCHAR, Dinamica VARCHAR);
Table EVICTIONS_1 created (or

C:\Users\andre\AppData\Local\Temp\ipykernel_26904\1437233484.py:59: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Nominations_raw.replace(r'(nenhum)', None, regex=True, inplace=True)


CREATE TABLE IF NOT EXISTS BIGBROTHERBRASIL.WIKIPEDIA.NOMINATIONS_17 (Semana VARCHAR, Lider VARCHAR, Anjo VARCHAR, Imunizado VARCHAR, Big_Fone VARCHAR, Indicado_Big_Fone VARCHAR, Indicado_Lider VARCHAR, Indicado_Casa VARCHAR, Edicao VARCHAR, Dinamica VARCHAR);
Table NOMINATIONS_17 created (or already exists).
Successfully uploaded 11 rows to NOMINATIONS_17
CREATE TABLE IF NOT EXISTS BIGBROTHERBRASIL.WIKIPEDIA.INDIVIDUAL_NOMINATIONS_17 (Semana VARCHAR, Emilly VARCHAR, Vivian VARCHAR, Ieda VARCHAR, Marcos VARCHAR, Marinalva VARCHAR, Ilmar VARCHAR, Daniel VARCHAR, Romulo VARCHAR, Roberta VARCHAR, Pedro VARCHAR, Elis VARCHAR, Manoel VARCHAR, Luiz_Felipe VARCHAR, Mayara VARCHAR, Gabriela_Flor VARCHAR, Mayla VARCHAR, Antonio VARCHAR, Edicao VARCHAR, Dinamica VARCHAR);
Table INDIVIDUAL_NOMINATIONS_17 created (or already exists).
Successfully uploaded 14 rows to INDIVIDUAL_NOMINATIONS_17
CREATE TABLE IF NOT EXISTS BIGBROTHERBRASIL.WIKIPEDIA.EVICTIONS_17 (Semana VARCHAR, Notas VARCHAR, Expulso 

C:\Users\andre\AppData\Local\Temp\ipykernel_26904\1437233484.py:59: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Nominations_raw.replace(r'(nenhum)', None, regex=True, inplace=True)


CREATE TABLE IF NOT EXISTS BIGBROTHERBRASIL.WIKIPEDIA.NOMINATIONS_21 (Semana VARCHAR, Lider VARCHAR, Anjo VARCHAR, Imunizado VARCHAR, Big_Fone VARCHAR, Indicado_Big_Fone VARCHAR, Indicado_Contragolpe VARCHAR, Indicado_Lider VARCHAR, Indicado_Casa VARCHAR, Bate_e_Volta_Indicados VARCHAR, Bate_e_Volta_Salvo VARCHAR, Edicao VARCHAR, Dinamica VARCHAR);
Table NOMINATIONS_21 created (or already exists).
Successfully uploaded 18 rows to NOMINATIONS_21
CREATE TABLE IF NOT EXISTS BIGBROTHERBRASIL.WIKIPEDIA.INDIVIDUAL_NOMINATIONS_21 (Semana VARCHAR, Juliette VARCHAR, Camilla VARCHAR, Fiuk VARCHAR, Gilberto VARCHAR, Pocah VARCHAR, Arthur VARCHAR, Viih_Tube VARCHAR, Joao_Luiz VARCHAR, Caio VARCHAR, Thais VARCHAR, Rodolffo VARCHAR, Sarah VARCHAR, Carla VARCHAR, Projota VARCHAR, Lumena VARCHAR, Karol VARCHAR, Nego_Di VARCHAR, Arcrebiano VARCHAR, Lucas VARCHAR, Kerline VARCHAR, Edicao VARCHAR, Dinamica VARCHAR);
Table INDIVIDUAL_NOMINATIONS_21 created (or already exists).
Successfully uploaded 20 row